In [13]:
from dataclasses import dataclass
from typing import Tuple, NewType

import keras
import numpy as np
import numpy.typing as npt


@dataclass
class Dataset:
    images: npt.NDArray[np.uint8]
    labels: npt.NDArray[np.uint8]


TrainDataset = NewType("TrainDataset", Dataset)
TestDataset = NewType("TestDataset", Dataset)


def fetch_datasets() -> Tuple[TrainDataset, TestDataset]:
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_dataset = TrainDataset(Dataset(images=train_images, labels=train_labels))
    test_dataset = TestDataset(Dataset(images=test_images, labels=test_labels))

    return train_dataset, test_dataset


train_dataset, test_dataset = fetch_datasets()

__main__.Dataset

In [18]:
import keras
import tensorflow as tf


def train_model(dataset: TrainDataset, epochs: int = 5) -> keras.Model:
    # Scale the values to 0.0 to 1.0
    train_images = dataset.images / 255.0

    # Reshape for feeding into the model
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)

    # Define the model architecture.
    model = keras.Sequential(
        [
            keras.layers.Conv2D(
                input_shape=(28, 28, 1),
                filters=8,
                kernel_size=3,
                strides=2,
                activation="relu",
                name="Conv1",
            ),
            keras.layers.Flatten(),
            keras.layers.Dense(10, name="Dense"),
        ]
    )
    model.summary()

    # Fit the model to the training data.
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    model.fit(train_images, dataset.labels, epochs=epochs)

    return model


model = train_model(train_dataset, epochs=1)
model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten_3 (Flatten)         (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13610 (53.16 KB)
Trainable params: 13610 (53.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5314 - sparse_categorical_accuracy: 0.8141


In [ ]:
from typing import Dict

def evaluate_model(test_dataset: TestDataset, model: keras.Model) -> Dict[str, float]:
    test_images = test_dataset.images / 255.0
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

    _, test_acc = model.evaluate(test_images, test_dataset.labels)

    return {"accuracy": test_acc}


In [4]:
from fashion_model.prepare import fetch_dataset

#fetch_dataset(output_path="_local")
fetch_dataset(output_path="s3://data/test")

In [12]:
import fsspec

fsspec.AbstractFileSystem

AttributeError: module 'fsspec' has no attribute 'Filesystem'

In [8]:
train_labels.dtype

dtype('uint8')

In [9]:
import numpy as np
import numpy.typing as npt

npt.NDArray[np.float64]

numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]]